## Dynamic risk parity portfolio with independent components

### Exploring the feasibility

In [3]:
eur_usd <- readxl::read_excel("score_data.xlsx", sheet = 1, 
col_types = c("text", "numeric", "numeric", "numeric", "numeric"))
eur_usd$date <- as.Date(eur_usd$date)
eur_usd <- eur_usd[order(eur_usd$date),]

gbp_usd <- readxl::read_excel("score_data.xlsx", sheet = 2,
col_types = c("text", "numeric", "numeric", "numeric", "numeric"))
gbp_usd$date <- as.Date(gbp_usd$date)
gbp_usd <- gbp_usd[order(gbp_usd$date),]


data <- as.data.frame(cbind(eur_usd$close, gbp_usd$close))  
data <-cbind(eur_usd$date, data) 
colnames(data) <- c("date", "euro", "gbp")


returns <- as.data.frame(cbind(diff(log(data$euro)), diff(log(data$gbp))))
returns <- cbind(data$date[2:4956], returns)
colnames(returns) <- c("date", "euro", "gbp")
returns$euro <- returns$euro - mean(returns$euro)
returns$gbp <- returns$gbp - mean(returns$gbp)

use_data <- cbind(returns$euro, returns$gbp)

